<a href="https://colab.research.google.com/github/EricTheProgrammer342/Homework1_BigData/blob/main/Homework_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.utils import to_categorical

#from google.colab import drive
#drive.mount('/content/drive')
from google.colab import files
uploaded = files.upload()


import re
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import os

#data = pd.read_csv('Sentiment.csv')
data = pd.read_csv('Sentiment.csv')


# Keeping only the neccessary columns
data = data[['text','sentiment']]

data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

embed_dim = 128
lstm_out = 196
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
# print(model.summary())

labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

batch_size = 32
model = createmodel()

#Amendment
model.save('sentiment_model.h5')
from tensorflow.keras.models import load_model
loaded_model = load_model('sentiment_model.h5')

#Amendment
collection = "A lot of good things are happening. We are respected again throughout the world and that's a great thing @realDonaldTrump"
collection = collection.lower()
collection = re.sub('[^a-zA-Z0-9\s]', '', collection)
collectionsequence = tokenizer.texts_to_sequences([collection])
collectionpadded = pad_sequences(collectionsequence, maxlen=X.shape[1])

#Amendment
Aspect1 = loaded_model.predict(collectionpadded)
Aspect1label = np.argmax(Aspect1, axis=1)
Aspect1sentiment = labelencoder.inverse_transform(Aspect1label)
print(f"Overall Sentiment in regards to data: {Aspect1sentiment[0]}")

model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)




Saving Sentiment.csv to Sentiment (4).csv


<ipython-input-7-f016e02b2fe9>:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row[0] = row[0].replace('rt', ' ')
<ipython-input-7-f016e02b2fe9>:33: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  row[0] = row[0].replace('rt', ' ')
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 866ms/step
Overall Sentiment in regards to data: Neutral
291/291 - 26s - 89ms/step - accuracy: 0.6407 - loss: 0.8345
144/144 - 5s - 32ms/step - accuracy: 0.6544 - loss: 0.7689
0.7688892483711243
0.6544342637062073
['loss', 'compile_metrics']
